In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
pd.set_option('display.max_rows', None)
data = pd.read_csv('Data_SEM_clear_v1.csv')
df=pd.DataFrame(data)
feature_names = df.keys()
all_feature_names= list(feature_names[2::])
num_feature_names = feature_names[5::]


x_num = df.iloc[:,5::]
standardScaler = StandardScaler()
x_num = pd.DataFrame(standardScaler.fit_transform(x_num),columns=num_feature_names)
x_cat = pd.DataFrame(df.iloc[:,2:5])
df_x = pd.concat([x_cat,x_num], axis=1)
df_x_orig = df_x.copy()
all_feature_names_orig = all_feature_names.copy()



In [14]:
from sklearn.metrics import precision_score,recall_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_recall_curve

y =  df['HOMA_IR']
model = RandomForestClassifier()
highest = 0
highest_index = 0
highest_y_test = 0
highest_y_pred = 0
for i in range(len(all_feature_names)):
    x = pd.DataFrame(df_x.iloc[:,i])
    # print(x.iloc[i].index)
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2,random_state=30)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    test_acc = accuracy_score(y_test, y_pred).round(7)
    if test_acc > highest:
        highest = test_acc
        highest_index = i
        highest_y_test = y_test
        highest_y_pred = y_pred

cm = confusion_matrix(highest_y_test, highest_y_pred)
print(pd.DataFrame(cm))
print(classification_report(highest_y_test, highest_y_pred))
print('特徵:',all_feature_names[highest_index])
print('正確率:', highest)        




    0   1
0  23  19
1  10  34
              precision    recall  f1-score   support

           0       0.70      0.55      0.61        42
           1       0.64      0.77      0.70        44

    accuracy                           0.66        86
   macro avg       0.67      0.66      0.66        86
weighted avg       0.67      0.66      0.66        86

特徵: BUN
正確率: 0.6627907


In [15]:
df_x = df_x.drop(columns = all_feature_names[highest_index])
all_feature_names.pop(highest_index)
second = 0
second_index = 0
second_y_test = 0
second_y_pred = 0
old_feature = df_x_orig.iloc[:,highest_index]
for i in range(len(all_feature_names)):
    new_feature = df_x.iloc[:,i]
    x = pd.concat([old_feature,new_feature], axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2,random_state=30)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    test_acc = accuracy_score(y_test, y_pred).round(7)
    if test_acc > second:
        second = test_acc
        second_index = i
        second_y_test = y_test
        second_y_pred = y_pred

cm = confusion_matrix(second_y_test, second_y_pred)
print(pd.DataFrame(cm))
print(classification_report(second_y_test, second_y_pred))
print('特徵:',all_feature_names[second_index])
print('正確率:', second)   


    0   1
0  23  19
1   8  36
              precision    recall  f1-score   support

           0       0.74      0.55      0.63        42
           1       0.65      0.82      0.73        44

    accuracy                           0.69        86
   macro avg       0.70      0.68      0.68        86
weighted avg       0.70      0.69      0.68        86

特徵: BMI
正確率: 0.6860465
